# 초기세팅

## 모듈 임포트

In [1]:
from typing import List, Dict, Optional, TypedDict, Any
from fastapi import FastAPI , Body ,HTTPException, APIRouter, Query 
import torch
import ast
import json
import uuid
import numpy as np
import psycopg2
from pgvector.psycopg2 import register_vector
from sentence_transformers import SentenceTransformer
from google.genai import types
from fastapi.middleware.cors import CORSMiddleware
import os
from langgraph.graph import StateGraph, END

## 모델 로딩

### 재미나이 로딩

In [2]:
# API 키 로딩
keys = []
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                keys.append(value)

# API 키 로드 및 환경변수 설정
load_api_keys('api_key.txt')

In [3]:
def keyChange():
    global client
    from google import genai
    tt = keys.pop(0)
    keys.append(tt)
    client = genai.Client(api_key=tt)

keyChange() #무료 키 다썻으면 다른 키로 로딩하기

### 백터DB & 임베딩 모델 로딩

In [4]:
model_vector = SentenceTransformer('jhgan/ko-sroberta-multitask')
# 2. 원격 pgvector 연결 설정
def get_db_connection():
    db_info= dict()
    with open("db_info.txt","r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=",1)
                db_info[key] = value

    conn = psycopg2.connect(
        host=db_info['host'],
        database=db_info['database'],
        user=db_info['user'],
        password=db_info['password'],
        port=db_info['port']
    )
    register_vector(conn)
    return conn


## State

In [5]:
class State(TypedDict, total = False):
    txt: str  #front
    ep_num : int  #front 
    subtitle : str #front
    check : dict
    work_id : int    #front
    user_id : str    #front
    인물 : dict
    세계 : dict
    장소 : dict
    사건 : dict
    물건 : dict
    집단 : dict
    


# 기능

## 백터 DB

### 임베딩 백터 : 택스트 데이터를 임베딩 벡터로 바꿔서 반환(768차원)

In [6]:
def get_vector(text):
    return model_vector.encode([text])[0]

### 백터 DB 검색(유사도)  
(카테고리 or *(전부), 유저 질문, user_id, work_id, 유사도 몇까지 출력할지, 몇개 출력할지)

In [7]:

def userQuestion(category,user_query,user_id,work_id,sim,limit):  # 태그(*,인물,장소,사건,세계,등)
    conn = get_db_connection()
    cur = conn.cursor()
    query_vector = get_vector(user_query)
    search_sql = """
    SELECT * FROM (
        SELECT 
            id,
            setting, 
            ep_num,
            1 - (embedding <=> %s::vector) AS similarity 
        FROM lorebooks 
        WHERE embedding IS NOT NULL 
            AND %s = ANY(user_id)
            AND work_id = %s
            AND (%s = '*' OR category = %s)
    ) sub
    WHERE similarity >= %s  -- 여기서 원하는 유사도 문턱값을 설정 (예: 0.3 이상)
    ORDER BY similarity DESC  
    LIMIT %s;
    """
    cur.execute(search_sql, (query_vector,user_id,work_id,category,category,sim,limit))
    result = cur.fetchall()
    cur.close()
    conn.close()
    return result

In [8]:
# userQuestion("인물","케인과 적대하는 인물","ssssssss",15,0.3,10)

### DB 검색(일반)  
(카테고리, 검색할 대상, user_id, work_id, 유사도 몇까지 출력할지, 몇개 출력할지)

In [9]:
def select(category,keyword,user_id,work_id): #태그,키워드,타이틀,작가 4개 다있어야함
    conn = get_db_connection()
    cur = conn.cursor()
    search_sql= """
    SELECT 
        id,
        setting,
        ep_num
    FROM lorebooks 
    WHERE %s = ANY(user_id)
        AND work_id = %s
        AND category = %s
        AND keyword = %s
    """
    cur.execute(search_sql, (user_id,work_id,category,keyword))
    result = cur.fetchall()
    # print(result)
    cur.close()
    conn.close()
    return result

### DB 업로드(설정집 신규)

In [10]:
def lorebooks_insert(universe_id,work_id,user_id,setting):
    conn = get_db_connection()
    cur = conn.cursor()
    for tag, values in setting.items():
        for val in values:
            if not val:
                continue
            ep_num = val.pop('ep_num',[])
            keyword,data = val.popitem()
            embd = get_vector(f'{keyword} : {data}')
            try:
                insert_sql = """
                    INSERT INTO lorebooks ( universe_id, work_id, user_id, keyword, category, ep_num, setting, embedding,created_at,updated_at)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s,CURRENT_TIMESTAMP,CURRENT_TIMESTAMP)
                    """
                cur.execute(insert_sql, (
                        universe_id, 
                        work_id,
                        [user_id],
                        keyword,
                        tag,
                        ep_num,
                        json.dumps({keyword:data}), 
                        embd # psycopg2가 리스트 형태를 자동으로 vector 타입으로 변환합니다.
                    ))
                conn.commit()
                print("성공적으로 저장되었습니다!")
            except Exception as e:
                print(e)
                return "500 Internal Server Error"  # DB연결실패
    cur.close()
    conn.close()

# 수동 신규 설정집 업로드
def db_insert(universe_id, work_id, user_id, keyword, category, ep_num, setting):
    embd = get_vector(f'{setting}')
    conn = get_db_connection()
    cur = conn.cursor()
    insert_sql = """
    INSERT INTO lorebooks ( universe_id, work_id, user_id, keyword, category, ep_num, setting, embedding)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    cur.execute(insert_sql, (
        universe_id,
        work_id,
        user_id,
        keyword,
        category,
        ep_num,
        json.dumps(setting),
        embd # psycopg2가 리스트 형태를 자동으로 vector 타입으로 변환합니다.
    ))
    conn.commit()
    cur.close()
    conn.close()
    print("성공적으로 저장되었습니다!")

### DB 수정(설정집 결합)

In [11]:
def lorebooks_update(setting):
    for lore_id,result,ep_num in setting:
        conn = get_db_connection()
        cur = conn.cursor()
        new_vector = f"{result}"
        embedding_vector = get_vector(new_vector)
        update_sql = """
        UPDATE lorebooks 
        SET 
            ep_num = %s,
            setting = %s,
            embedding = %s,
            updated_at = CURRENT_TIMESTAMP
        WHERE id = %s;
        """
        try:
            cur.execute(update_sql, (ep_num,json.dumps(result), embedding_vector, lore_id))
            conn.commit()
            print(f"데이터가 성공적으로 수정되었습니다.")
        except Exception as e:
            conn.rollback()
            print(e)
            return (f"수정 실패: {e}")
        finally:
            cur.close()
            conn.close()


# DB 수동 수정
def db_update(lore_id,universe_id, work_id, user_id, keyword, category, ep_num, setting):
    embd = get_vector(f'{setting}')
    conn = get_db_connection()
    cur = conn.cursor()
    embedding_vector = get_vector(new_vector)
    update_sql = """
    UPDATE lorebooks 
    SET 
        universe_id = %s,
        work_id = %s,
        user_id = %s,
        keyword = %s,
        category = %s,
        ep_num = %s,
        setting = %s,
        embedding = %s,
        updated_at = CURRENT_TIMESTAMP
    WHERE id = %s;
    """
    try:
        cur.execute(update_sql, (universe_id, work_id, user_id, keyword, category, ep_num, json.dumps(setting), embd, lore_id))
        conn.commit()
        print(f"데이터가 성공적으로 수정되었습니다.")
    except Exception as e:
        conn.rollback()
        return (f"수정 실패: {e}")
    finally:
        cur.close()

### 연관인물 검색

In [12]:
def relationship_select(target,work_id,user_id): #태그,키워드,타이틀,작가 4개 다있어야함
    conn = get_db_connection()
    cur = conn.cursor() #쿼리문 확인하고 수정해야함.
    search_sql= """
    SELECT 
    
    -- 매칭되는 인물관계 객체만 JSON 형태로 출력
    jsonb_build_object(
        l.keyword, relation
    ) AS matching_relation
FROM 
    
    lorebooks l,
    jsonb_each(l.setting) AS person, -- 1단계: 인물명(Key)별로 분리
    jsonb_array_elements(person.value->'인물관계') AS relation -- 2단계: 인물관계 배열 전개
WHERE 
    l.work_id = %s  -- 입력받을 work_id
    AND %s = ANY(l.user_id)  -- 입력받을 user_id
    AND relation->>'대상이름' = %s -- 검색하려는 대상인물 이름
    AND l.category = '인물'"""
    cur.execute(search_sql, (work_id,user_id,target))
    result = cur.fetchall()
    # print(result)
    cur.close()
    conn.close()
    meinfo = select("인물",target,user_id,work_id)
    meinfo = [{target:meinfo[0][1][target]['인물관계']}]
    return result+meinfo


## LLM 

### 메시지 입력

In [13]:
count_change= 0
models = ["gemini-2.5-flash-lite", 'gemini-3-flash',"gemini-2.5-flash","gemma-3-27b-it"]
def modelchange():
    global mod
    mod = models.pop(0)
    models.append(mod)
modelchange()    
def genai(system_prompt,msg):
    global count_change
    try:
        if mod == "gemma-3-27b-it":
            response = client.models.generate_content(
                model=mod,
                contents=system_prompt+msg,
            )
            return response.text
        else:
            response = client.models.generate_content(
                model= mod,
                config=types.GenerateContentConfig(
                    system_instruction=system_prompt, # 여기에 시스템 프롬프트를 넣습니다.
                    temperature=0.1, # 데이터 추출이므로 낮게 설정
                ),
                contents=msg
            )
            return response.text
    except Exception as e:
        count_change +=1
        if count_change >=30:
            modelchange()
            count_change = 0
        keyChange()
        return genai(system_prompt,msg)


### 카테고리 추출

In [14]:
def Categories(state: State) -> State:
    system_prompt="""### Role
너는 소설 원문에서 설정 데이터를 누락 없이 추출하여 구조화하는 전문 데이터 분석가입니다.

### Task
당신은 '본문'의 글에서 설정집을 생성하기 쉽도록 키워드를 추출합니다. 
추출할 정보는 '인물, 세계, 장소, 사건, 물건, 집단'에 관한 내용이며, 설정집에 필요한 필수적인 정보들만 추출할 것입니다.
이를 딕셔너리{} 형태로 구조화하세요.

### Extraction Guidelines (카테고리별 정의)
- **인물** 
이름이 언급되거나 고유한 역할이 있는 등장인물(단순 엑스트라 제외)
- **세계**
마법 체계, 정치 시스템, 신화적 배경, 물리적 법칙 등 세계관을 이해하기에 필수적인 설정과 개념
- **장소**
사건이 벌어지는 지리적 배경, 건물 이름, 특정 지역
- **사건**
스토리에 영향을 끼칠 수 있는 구체적인 변화, 전투, 조약, 과거의 사건 등
- **물건**
고유 명칭이 있는 도구, 무기, 유물, 중요한 상징적 아이템
- **집단**
가문, 국가, 단체 등 2인 이상의 고유명칭을 가진 결사체

### Constraints
- 특정 주인공 중심의 요약을 하지 마세요. 등장하는 키워드를 분류하세요.
- 결과물에는 불릿 포인트, 볼드체, 추가 설명을 절대 포함하지 말고 순수 JSON/Dict 형식만 출력하세요.
- **본문에 명확히 나오지 않은 정보는 빈 배열 `[]`로 처리하세요.
- **수식어 제거**: 대상을 특정하기 위한 수식어(예: 'oooo의 정치체계','oo의 악마','oo왕국 기사단','oooo 제국')를 제외한 수식어는(예: '비시르 함장' X, '비시르' O) 모두 제거하고 핵심 키워드만 추출하세요.
- **개체명 표준화: 동일 대상을 지칭하는 이명(예: '몰락한 왕'과 '비에고','비에고 국왕'과 '비에고' > 두 경우 다 '비에고'로 출력)은 가장 공식적인 명칭 하나로 통합하세요.

### Output Format 
{
'인물':['인물1','인물2',...],
'세계':['규칙1','규칙2',...],
'장소':['장소1','장소2'],
'사건':['사건1','사건2'],
'물건':['물건1',....],
'집단':['집단1','집단2'],
}

"""
    msg = f"본문: {state.get('txt','')} \n 위 본문을 분석하여 '인물, 세계, 장소, 사건, 물건, 집단' 정보를 추출해줘."
    
    result = genai(system_prompt,msg)
    print("목록 추출 완료")
    print("=="*8)
    print(result)
    print("=="*8)
    try:
        st = result.index('{')
        ed = result.index('}')
        result = ast.literal_eval(result[st:ed+1])
    except:
        print("딕셔너리 변환 에러")
        # state = Categories(state)
    return {**state,
            'check' : result
           }

### 1.설정 추출 - 인물

In [15]:
def charter(state: State) -> State:
    lis = state.get('check')
    lis = lis['인물']
    txt = state.get('txt')
    chart = state.get('인물',{})
    for i in range(0, len(lis), 2):
        chunk = lis[i:i+2]  # 예: ['인물1', '인물2']
        names_str = ", ".join(chunk)
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 인물 설정집(Character Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 등장인물명 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
새로운 정보가 발견될 때마다 기존 설정과 대조할 수 있도록 매우 구체적이고 객관적인 수치와 묘사 위주로 추출해야 합니다.
각 인물별로 독립된 키를 가진 딕셔너리 형태로 출력해야 합니다
본문에서 해당 정보가 있다면 None값을 삭제하고 정보를 str형식으로 적으세요
# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
"""+"""   
# JSON_Output_Format
{ "인물명1" : 
    { "별명": "본문에 명시된 별명 (없으면 None)",
      "배경": "간략한 배경 스토리(없으면 None)",
      "종족": "종족 (없으면 None)",
      "연령": "나이 또는 연령대 (없으면 None)",
      "직업/신분": "신분 (없으면 None)",
      "소속집단/가문": "소속 (없으면 None)",
      "외형": "외모 묘사 (없으면 None)",
      "성격": ["성격 키워드1 (없으면 None)"],
      "기술/능력": [
        {"기술명": "능력 이름 (없으면 None)", "숙련도/강도": "수준 (없으면 None)", "상세효과": "효과설명 (없으면 None)"}
      ],
      "인물관계": [
        {"대상이름": "상대방 이름(없으면 None)", "관계": "관계 정의( 없으면 None"), "상세내용": "관계 설명(없으면 None)"}
      ]
    },
    "인물명2" : 
    { "별명": "본문에 명시된 별명 (없으면 None)",
      "배경": "간략한 배경 스토리(없으면 None)",
      "종족": "종족 (없으면 None)",
      "연령": "나이 또는 연령대 (없으면 None)",
      "직업/신분": "신분 (없으면 None)",
      "소속집단/가문": "소속 (없으면 None)",
      "외형": "외모 묘사 (없으면 None)",
      "성격": ["성격 키워드1 (없으면 None)"],
      "기술/능력": [
        {"기술명": "능력 이름 (없으면 None)", "숙련도/강도": "수준 (없으면 None)", "상세효과": "효과설명 (없으면 None)"}
      ],
      "인물관계": [
        {"대상이름": "상대방 이름(없으면 None)", "관계": "관계 정의( 없으면 None"), "상세내용": "관계 설명(없으면 None)"}
      ]
    }
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 만약 등장인물이 하나만 있을 경우에는 한 개체만 출력하세요.
- 제공한 등장인물 2명을 제외한 나머지는 추가적으로 출력하지 마세요."""
        
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식을 마크다운 없이 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)

            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                chart[k] = v
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
        
    return {**state,
           '인물' :chart
           }

### 2.설정추출 - 세계관

In [16]:
def rule(state: State) -> State:
    lis = state.get('check')
    lis = lis['세계']
    txt = state.get('txt')
    rule = state.get('세계',{})
    for i in range(0, len(lis), 4):
        chunk = lis[i:i+4]  # 예: ['인물1', '인물2']
        names_str = ", ".join(chunk)
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 세계관 설정집을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 세계관 키워드 [{names_str}] 각각의 정보을/를 본문 없이 작품 속 세계의 규칙을 이해할 수 있도록 300자 이내로 설명하는 문장을 만들어 JSON 포맷으로 출력하세요.

# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 요약은 가능하지만 본문의 내용을 바탕으로 추측 없이 문장을 생성하세요.
2. 연관된 이야기 없이 대상에 대한 내용만으로 짧게 요약하여 문장을 생성하세요.

"""+"""   
# JSON_Output_Format
{
"세계관 키워드1" : "세계관 1 설명",
"세계관 키워드2" : "세계관 2 설명",....
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 명확히 나오지 않은 정보는 None값 또는 빈 배열 `[]`로 처리하세요.
- 만약 세계관 키워드가 하나만 있을 경우에는 한 세계관만 출력하세요.
- 제공한 키워드를 제외한 나머지는 추가적으로 출력하지 마세요.
"""
        
        msg= f"규칙을 바탕으로 순수한 Json 텍스트 형식을 마크다운 없이 출력하세요."           
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)

            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                rule[k] = v
        except:
            print("딕셔너리 변환 에러")
            print("result")
            #state = charter(state)
    
    return {**state,
           '세계' :rule
           }

### 3.설정추출 - 장소

In [17]:
def place(state: State) -> State:
    lis = state.get('check')
    lis = lis['장소']
    txt = state.get('txt')
    place = state.get('장소',{})
    for i in range(0, len(lis), 3):
        chunk = lis[i:i+3]  
        names_str = ", ".join(chunk)
        system_prompt=  f"""# Role
귀하는 소설의 원문을 분석하여 장소 설정집(Place Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 장소 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
새로운 정보가 발견될 때마다 기존 설정과 대조할 수 있도록 매우 구체적이고 객관적인 수치와 묘사 위주로 추출해야 합니다.
본문에서 해당 정보가 있다면 None값을 삭제하고 정보를 str형식으로 적으세요

# 본문
{txt}

# extraction_rules
1. **Fact-Based**: 추측하지 말고 본문에 명시된 텍스트 그대로를 기반으로 추출하세요.
"""+"""   
# JSON_Output_Format
{"장소 키워드1": 
    { "별칭": "지역별칭 (없으면 None)"
      "분위기": "지역의 분위기(없으면 None)",
      "역사": "지역의 역사 또는 생성된 시기 (없으면 None)",
      "위치": "지역의 위치 (없으면 None)",
      "집단": ["해당 공간에 속한 소속 집단/가문(없으면 None)"],
      "하부지역": ["이 장소에 속한 다른 지역 (없으면 None)"],
      "작중묘사" :["해당 장소에 대한 묘사 (없으면 None)"]
    },
 "장소 키워드2": 
    { "별칭": "지역별칭 (없으면 None)"
      "분위기": "지역의 분위기(없으면 None)",
      "역사": "지역의 역사 또는 생성된 시기 (없으면 None)",
      "위치": "지역의 위치 (없으면 None)",
      "집단": ["해당 공간에 속한 소속 집단/가문(없으면 None)"],
      "하부지역": ["이 장소에 속한 다른 지역 (없으면 None)"],
      "작중묘사" :["해당 장소에 대한 묘사 (없으면 None)"]
    }
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 정보가 전혀 없는 항목만 None값으로 남겨두세요.
- 만약 장소 키워드가 하나만 있을 경우에는 한 장소만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요."""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
                    

        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                place[k] = v
        except:
            print("딕셔너리 변환 에러")
            print("result")
            #state = charter(state)
    
    return {**state,
           '장소' :place
           }

### 4.설정추출 - 사건

In [18]:
def event(state: State) -> State:
    lis = state.get('check')
    lis = lis['사건']
    txt = state.get('txt')
    event = state.get('사건',{})
    for i in range(0, len(lis), 5):
        chunk = lis[i:i+5]  
        names_str = ", ".join(chunk)
        system_prompt= f"""# Role
귀하는 소설의 원문을 분석하여 사건 설정집(Place Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.

# Task
제공된 소설 본문을 읽고, 미리 추출한 사건 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
설명에는 해당 사건에 관해 간략한 시점, 관련개체, 원인, 결과를 하나의 문장으로 요약하여 써야한다
만약 시점,관련개체,원인,결과중 존재하지 않는 사항이 있다면 해당 사항은 빼고 나머지 사항으로 한줄요약하여 써야한다.
# 본문
{txt}
"""+"""   
# JSON_Output_Format
{
"사건 키워드1": "해당 사건에 관한 짧은 설명",
"사건 키워드2": "해당 사건에 관한 짧은 설명"
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 만약 키워드가 하나만 있을 경우에는 한 사건만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요.
"""
        msg=f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                event[k] = v
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '사건' :event
           }

### 5.설정추출 - 아이템

In [19]:
def item(state: State) -> State:
    lis = state.get('check')
    lis = lis['물건']
    txt = state.get('txt')
    item = state.get('물건',{})
    
    for i in range(0, len(lis), 4):
        chunk = lis[i:i+4]  
        names_str = ", ".join(chunk)
        system_prompt = f"""# Role
귀하는 소설의 원문을 분석하여 아이템 설정(item Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.
본문에서 해당 정보가 있다면 None값을 삭제하고 정보를 str형식으로 적으세요

# Task
제공된 소설 본문을 읽고, 미리 추출한 아이템 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
summary에는 물건의 성능과 역할에 대해 간략하게 적어야 합니다.
character에는 해당 사건의 메인 인물을 적어야 합니다.

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{
"아이템 키워드1": {"설명":"해당 물건 관한 짧은 설명 (없으면 None)",
"관련인물" :["관련된인물 (없으면 None)"] },
"아이템 키워드2": {"설명":"해당 물건 관한 짧은 설명 (없으면 None)",
"관련인물" :["관련된인물 (없으면 None)"] }
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 관련인물은 사건에 등장하는 모든 인물이 아닌 아이템과 관련된 주역만 적어야 합니다. 주역이 없을 경우 빈 배열 '[]'로 처리하세요.
- 만약 키워드가 하나만 있을 경우에는 한 물건만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요.
"""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                item[k] = v
        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '물건' :item
           }

### 6.설정추출 - 단체

In [20]:
def group(state: State) -> State:
    lis = state.get('check')
    lis = lis['집단']
    txt = state.get('txt')
    group = state.get('집단',{})
    
    for i in range(0, len(lis), 3):
        chunk = lis[i:i+3]
        names_str = ", ".join(chunk)
        system_prompt=f"""# Role
귀하는 소설의 원문을 분석하여 집단 설정(group Profile)을 생성하는 전문 편집자이자 데이터 분석가입니다.
본문에서 해당 정보가 있다면 None값을 삭제하고 정보를 str형식으로 적으세요

# Task
제공된 소설 본문을 읽고, 미리 추출한 집단 키워드 [{names_str}] 각각의 정보를 분석하여 지정된 JSON 포맷으로 출력하세요. 
summary에는 집단의 역사와 목적 대해 간략하게 적어야 합니다.
character에는 해당 집단에 속해있는 인물을 적은 후 집단에서 역할을 괄호'()'안에 적으세요.

# 본문
{txt}
"""+"""   
# JSON_Output_Format
{
"집단 키워드1": {"설명":"집단에 관한 짧은 설명(없으면 None)",
"구성원" :["구성원이름(역할)(없으면 None)"] },
"집단 키워드2": {"설명":"집단에 관한 짧은 설명(없으면 None)",
"구성원" :["구성원이름(역할)(없으면 None)"] },
}

# Constraints
- 응답은 반드시 순수한 JSON 형식이어야 하며, 다른 설명 문구는 포함하지 마세요.
- 본문에 정보가 전혀 없는 항목만 None값으로 남겨두세요.
- 구성원은 대상을 특정하기 위한 수식어(예: '데마시아의 정치체계','총의 악마','OO왕국 기사단')를 제외한 수식어는(예: '비시르 함장' X, '비시르' O) 모두 제거하여 넣으세요.
- 만약 키워드가 하나만 있을 경우에는 한 집단만 출력하세요.
- 제공한 키워드를 제외한 나머지는 임의로 추가하여 출력하지 마세요.
"""
        msg = f"규칙을 바탕으로 순수한 Json 텍스트 형식으로 출력하세요."
        result = genai(system_prompt,msg)
        try:
            result = result.replace("json", "").replace("```", "").strip()
            print(result)
            result = json.loads(result)
            print("변환 성공!")
            for k, v in result.items():
                group[k] = v

        except:
            print("딕셔너리 변환 에러")
            print(result)
            #state = charter(state)
    
    return {**state,
           '집단' :group
           }

### 관계도 다이어그램 출력

In [21]:
def Relationship(rel):
    system_prompt=f"""### Role
당신은 소설의 설정집을 보고 인물의 관계도를 코드로 출력해주는 전문가입니다.

### Task
당신은 인물의 관계도 데이터를 받아 Mermaid 코드 형식으로 생성해야 합니다.


### Constraints
- 만약 관계가 None일 경우 나머지 정보로 추측하여 None을 지우고 교체하세요.
- 결과물에는 불릿 포인트, 볼드체,제목,형식 추가 설명을 절대 포함하지 말고 순수 Mermaid 코드 형식만 출력하세요.
- ``` 나 형식이름과 같은 출력을 포함하지 말고 Mermaid 코드만 출력하세요

"""
    msg = f"{rel}인물 관계 데이터가 이렇게 있을 때 mermaid 코드로 출력해줘."
    
    result = genai(system_prompt,msg)
    print("관계도 출력 완료")
    print(result)
    return result



### 추출한 설정 검수 & 분류

In [22]:
def comparison(state: State):
    check = state.get('check')
    work_id = state.get('work_id')
    user_id = state.get('user_id')
    uplist ={'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    editlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    errorlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    oldlist = {'인물':[],'세계':[],'장소':[],'사건':[],'물건':[],'집단':[]}
    for tag,val in check.items():
        for keyword in val:
            emb = select(tag,keyword,user_id,work_id)
            if (len(emb)==0):
                uplist[tag].append({keyword:state.get(tag).get(keyword),"ep_num":[state.get("ep_num")]}) 
                continue
            oldlist[tag].append(emb)
            i = state.get(tag).get(keyword)
            system_prompt= f"""#### 역할: 소설 설정 검수 전문가
        
### 상황
하나의 대상에 대하여 서로 다른 에피소드에서 뽑아온 설정이 있습니다. 해당 두 설정을 확인하고 설정이 어긋나는 경우가 존재하는지 찾아보세요.

### 분석할 설정
기존설정 : {emb[0][1]}

신규설정 : {i}

### 요청 사항
2. **상세 분석:** - **인과관계/상세화:** 
   - **결합:** 한쪽이 원인이고 다른 쪽이 결과이거나, 한쪽이 다른 쪽에서 없는 설명인 경우 '결합'로 판단합니다.
   - **충돌:** 예외 없이 물리적으로 불가능한 경우에만 '충돌'로 판단하세요.
    -**일치:** 설정이 추가되거나 보강되는 부분 없이 동일한 경우에는 '일치'로 판단하세요.
    
### 출력 형식
[결과: 충돌/결합/일치]
[판단사유: 판단사유]
        """
            msg = f"이거 설정이 맞을까?"     
            result = genai(system_prompt,msg)
            if "[결과: 충돌]" in result:
                errorlist[tag].append({keyword:result,"신규설정":i})
            elif "[결과: 결합]" in result:
                editlist[tag].append(emb)
            print(result)
    editlist = edit_setting(editlist,state)
    return {'충돌': errorlist, '설정 결합' : editlist,
            "신규 업로드": uplist, "기존설정" : oldlist}

In [23]:
def edit_setting(editlist, state:State):
    lis = []
    for tag,value in editlist.items():
        for emb in value:            
            vctid,oldset,episode = emb[0]
            keyword = next(iter(oldset))
            system_prompt= f"""#### 역할: 소설 설정 작성 전문가
        
### 상황
하나의 대상에 대하여 서로 다른 에피소드에서 뽑아온 설정이 있습니다. 해당 두 설정을 확인하고 두 설정을 결합하세요.

### 분석할 설정
기존설정 : {oldset}

신규설정 : {keyword} : {state.get(tag).get(keyword)}

### 제한 사항
-기존 설정과 신규 설정의 시점 차이가 있을 경우, 시간상으로 나중에 일어난 일으로 업데이트하세요.
(예: 16세일때 외형묘사가 흑발, 18세일때 외형묘사가 금발이라면 외형을 금발으로 업데이트)
-서로 상충되는 부분이 아니라 보강되는 부분이라면 두 설정을 결합하세요.
(예: 기존 설정- 외형: 근육질, 신규 설정 -외형 : 흑발, > 결합 설정- 외형 : 근육질, 흑발
     설정1 -인물관계: 동료, 설정2-인물관계:오랜 친구 > 결합설정 - 인물관계 : 동료이자 오랜 친구)

### 출력 형식
설정 형식과 동일
"""
            msg = f"두 설정을 결합해줄래"     
            result = genai(system_prompt,msg)
            try:
                result = result.replace("json", "").replace("```", "").strip()
                result = json.loads(result)
            except:
                print("딕셔너리 변환 에러 : edit_setting")
                print(result)
                #state = charter(state)
            print(result)
            episode.append(state.get('ep_num',0))
            lis.append([vctid,result,episode])
    return lis

## 원문&설정 읽기 쓰기

In [24]:
def save_text(user_id, work_id, ep_num, txt):
    try:
        save_dir = f"./save_original/{user_id}/{work_id}"
        os.makedirs(save_dir, exist_ok=True)
        file_path = os.path.join(save_dir, f"{ep_num}.txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(txt)
            f.close()
        return f"./save_original/{user_id}/{work_id}/{ep_num}.txt"
    except Exception as e:
        print("error occured: ", e)
        return f"failed"

def open_text(user_id, work_id, ep_num) -> str:
    try:
        file_path = f"./save_original/{user_id}/{work_id}/{ep_num}.txt"
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e: 
        print("error occured: ", e)
        return "failed"
        
def save_json(user_id, work_id,lore_id, data):
    try:
        save_dir = f"./save_setting/{userid}/{work_id}"
        os.makedirs(save_dir, exist_ok=True)
        file_path = os.path.join(save_dir, f"{lore_id}.json")
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        return f"./save_setting/{userid}/{work_id}/{lore_id}.json"
    except Exception as e:
        print("error occured: ", e)
        return "failed"

def open_json(user_id, work_id,lore_id) :
    try:
        file_path = f"./save_original/{user_id}/{work_id}/{lore_id}.json"
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e: 
        print("error occured: ", e)
        return "failed"

## Rang Graph

### 카테고리

In [25]:
cat_builder = StateGraph(State)
cat_builder.add_node("Categories",Categories)

cat_builder.set_entry_point("Categories")
cat_builder.add_edge("Categories",END)

cat = cat_builder.compile()

### 상세설정

In [26]:
builder = StateGraph(State)
builder.add_node("charter",charter)
builder.add_node("rule",rule)
builder.add_node("place",place)
builder.add_node("event",event)
builder.add_node("item",item)
builder.add_node("group",group)

builder.set_entry_point("charter")
builder.add_edge("charter","rule")
builder.add_edge("rule","place")
builder.add_edge("place","event")
builder.add_edge("event","item")
builder.add_edge("item","group")
builder.add_edge("group",END)

detail = builder.compile()


# FastAPI

In [27]:
app = FastAPI()
users : Dict[str, State] = {}
origins = ["https://ai0917-kt-aivle-shool-8th-bigprojec-eta.vercel.app",
          "hppt://localhost:8080"]
app.add_middleware(
CORSMiddleware,
allow_origins=origins,
allow_credentials=True,
allow_methods=["GET", "POST", "PUT", "DELETE"],  
allow_headers=origins,
)
@app.get("/")
def connect_check():
    return "AI 서버와 연결되었습니다."

#원문 저장 
@app.post("/novel_save")
async def process_novel_save(data : dict = Body(...)):
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    ep_num = data.get("ep_num")
    txt = data.get("txt")
    return save_text(user_id, work_id, ep_num, txt)

# 원문 읽기
@app.get("/novel_read")
async def process_novel_read(
    user_id: str = Query(...), 
    work_id: str = Query(...), 
    ep_num: int = Query(...)
):
    # 기존 코드의 txt 변수가 정의되어 있지 않아 에러가 날 수 있으니 확인이 필요해요!
    return open_text(user_id, work_id, ep_num)

#설정 저장
@app.post("/setting_save")
async def process_setting_save(data : dict = Body(...)):
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    lore_id = data.get("lore_id")
    setting = data.get("data")
    return save_json(user_id,work_id,lore_id,setting)

# 설정 읽기
@app.get("/setting_read")
async def process_setting_read(
    user_id: str = Query(...), 
    work_id: str = Query(...), 
    lore_id: str = Query(...)
):
    return open_json(user_id, work_id, lore_id)

#카테고리 추출 
@app.post("/categories")
async def process_cat(data : dict = Body(...)):
    global current_state
    initial_state : State = {
        "txt": open_text(data.get("user_id"),data.get("work_id"),data.get("ep_num")),
        "ep_num" : data.get("ep_num"),
        "subtitle" : data.get("subtitle"),
        "check" : dict(),
        "work_id" : data.get("work_id"),
        "user_id" : data.get("user_id"),
        "인물" : dict(),
        "세계" : dict(),
        "장소" : dict(),
        "사건" : dict(),
        "물건" : dict(),
        "집단" : dict(),
    }
    if initial_state['txt'] == "failed":
        return "Error 원문이 저장되어 있지 않습니다."
    work_id = initial_state.get('work_id')
    result = cat.invoke(initial_state)
    users[work_id] = result
    return result.get("check",{})

@app.post("/setting") # 임베딩 벡터 반환하는 코드도 작성하기
async def process_get_setting(data : dict = Body(...)):
    work_id = data.get("work_id")
    categories = data.get("check", [])
    if work_id not in users:    # 존재하는 세션인지 확인
        raise HTTPException(status_code=404, detail="세션이 만료되었거나 존재하지 않습니다.")
    user_state = users[work_id]  #기존의 데이/터 가져오기
    user_state["check"] = categories
    user_state = detail.invoke(user_state)
    result = comparison(user_state)
    return result 

@app.post("/comparison") 
async def process_get_comparison(data : dict = Body(...)):
    result = comparison(data)
    return result 

@app.post("/get_vector")
async def process_get_vector(data : dict = Body(...)):
    original = data.get("original")
    txt_original = f"{original}"
    return get_vector(txt_original)

#유사도 기반 vectorDB 검색
@app.post("/userQ")
async def process_userQ(data : dict = Body(...)):
    category = data.get("category")
    user_query = data.get("user_query")
    user_id = data.get("user_id")
    work_id = data.get("work_id")
    sim = data.get("sim")
    limit = data.get("limit")
    return userQuestion(category,user_query,user_id,work_id,sim,limit)


@app.post("/dbupsert")
async def process_dbupsert(data : dict = Body(...)):
    work_id = data.get('work_id')
    user_id = data.get('user_id')
    universe_id = data.get('universe_id')
    setting = data.get('setting')
    try:
        insert_setting = setting.get('신규 업로드')
        lorebooks_insert(universe_id,work_id,user_id,insert_setting)
        edit_settings = setting.get('설정 결합')
        lorebooks_update(edit_settings)
        users.pop(work_id,None)
    except Exception as e:
        return e
    return "성공"

@app.post("/relationship")
async def process_relationship(data : dict = Body(...)):
    work_id = data.get('work_id')
    user_id = data.get('user_id')
    target = data.get('target',"")
    result = relationship_select(target,work_id,user_id)
    result = Relationship(result)
    return result


# 신규

In [28]:
# a = """그들은 절벽 아래에 있는 벌집 모양의 동굴들을 향해 모래 경사면을 올라갔다. 햇빛이 너무나 강렬했기에 절벽 밑의 연보라색 그림자 안으로 들어가자 지하 저장고에 들어간 것처럼 시원했다.

# 나쿠리의 인터페이스에서 메시지 알림음이 울렸다. 나쿠리는 메시지를 확인하기 위해 잠시 자리를 피했다. 케인과 슬링트루퍼 부대는 그림자 아래서 대기했다. 케인은 수백만 년 동안 사막의 바람에 침식된 동굴 입구들을 바라보았다.

# 그리고 또다시, 그는 무언가를 들었다.

# 목소리였다. 명확한 단어가 아닌 웅얼거림이었다. 그는 대기 중인 대원들을 두고 동굴을 향해 걸어갔다. 동굴 속의 어둠이 그를 향해 조용히 입을 벌리고 있었다.

# 아무것도 없었다.

# 그리고 다시 웅얼거림이 들렸다. 반은 웅얼거림이었고, 반은 키득거림이었다. 가장 가까이 있는 동굴 안일까? 무언가가 어둠 속에서 숨죽인 채 그를 보며 웃고 있는 것 같았다.

# 케인은 얼굴을 찌푸리고 한 걸음 더 다가갔다.

# 그의 인터페이스에서 알림음이 울렸다. "케인이다." 그는 작게 응답했다.

# 그의 왼쪽 눈에 '프랙털 쉬어'의 함교에 있는 바서르 함장이 흐릿하게 보였다. "각하, 경보 차원에서 연락드렸습니다. 슬링스피드 이하의 속도로 아이오난 공역을 향해 이동 중인 미약한 신호 반응을 감지했습니다."

# "미약한 신호 반응이라고?"

# "확실한 데이터도 없고, 정확한 위치도 파악할 수 없습니다. 유령처럼 말이죠."

# "보여줘."

# 바서르는 그의 인터페이스에 전함 주 탐지 시스템의 실시간 영상을 전송했다. 명확한 질량도, 밀도도 없었다. 사실, 탐지관들이 일반적으로 배경 왜곡이라 생각하고 무시할 정도의 데이터 수차였다. 하지만 지상에 오디널을 홀로 내려보낸 바서르는 극도로 신중할 수밖에 없었다.

# "범죄자들이 쓰는 은폐장이군." 케인이 말했다.

# "저도 그렇게 생각했습니다." 바서르는 말을 이었다. "특히 신디케이트 쪽에서 많이 사용하죠. 밀수범 검거 작전 중에 여러 번 봤습니다. 만약 은폐장이라면 꽤 훌륭하군요."

# "맞아. 훌륭한 성능이군."

# "접근을 차단할까요?"

# "아니."

# "그러면 좀 더 가까이 접근할까요? 만약을 대비해서 아이오난을 포격 거리 내에 두어야..."

# "아니야, 함장. 이곳에서 뭔가가 벌어지고 있어. 불온 분자들이 무언가를 되찾기 위해 왔을지도 모르고. 아직 그 무언가를 교환하기 전일지도 몰라. 그자들이 뭔가를 가지러 온 거라면 쫓아내지 말고 정체를 드러낼 때까지 기다리도록 하지."

# "각하께서 확신하신다면."

# "확신하네. 어떤 자들일지 두고 보자고. 이곳에 엄청난 정보가 있을지도 몰라."

# 케인은 연결을 끊고, 걸어오는 나쿠리를 향해 몸을 돌렸다.

# "어디 보자," 케인이 말했다. "미약한 신호 반응이겠군?"

# 나쿠리는 고개를 끄덕였다. "'프랙털 쉬어' 쪽에서도 알고 있나 보군. 양쪽 전함 사이의 성계 내곽은 우리가 잘 감시하고 있어. 그러니 별일 아닐 거야."

# "'젠틀 리마인더' 쪽에는 현 위치에서 대기하라고 명령했겠지?"

# "아무 행동도 취하지 말라고도 했네." 나쿠리가 웃으며 대답했다. "네 방식은 잘 알아. 어떤 놈들일지 얼굴 한번 보자고."

# 나쿠리는 돌아서서 가장 거대한 동굴 입구로 가는 마지막 경사면 구간으로 케인을 이끌었다. 슬링트루퍼들이 그 뒤를 따랐다. 케인은 여유와 만족감을 느꼈다. 나쿠리처럼 신뢰할 수 있는 현명한 자와 함께 작전을 벌이는 것은 즐거운 일이다. 지금까지 그래 왔듯이 그들은 멋진 팀이었다.

# 그는 마음속 깊은 곳에 숨은 이상한 느낌을 무시했다. 그것은 단순하고 정상적인 공포였으며, 잠재적인 위험이 도사리고 있는 불안한 상황을 처리할 때 드는 긴장감이었다.

# 그에게는 이런 사사로운 감정에 신경 쓰고 있을 시간이 없었다.


# 죄수들은 절벽의 바깥 동굴에 갇혀 있었다. 나쿠리의 슬링트루퍼들이 죄수들에게 에너지 족쇄를 채웠고, 솔리파스라는 장교가 지휘하는 2분대가 그들을 감시하고 있었다.

# 개성 강한 다양한 종족의 죄수들은 더럽고 닳아빠진 옷을 걸치고 있었다. 일부는 심문 중 구타당하기도 했다. 케인은 오라를 이용한 생체강화가 모두 제거되었다는 사실을 눈치챘다. 그 과정은 보기 싫은 상처를 남겼다.

# 기사단이라면 종파 그 이상도 아니었다. 불온 종자들이 모인 유사 신비주의 연합은 자신들이 오라의 진정한 '수호자'이며 그 물질에 대해 누구보다 깊이 이해하며, 다른 단체가 그것을 남용하지 못하도록 보호한다고 믿었다. 케인은 긴 복무 경력만큼이나 많은 기사단원들을 심문했고, 그들 대부분은 터무니없는 자들이었다. 기사단원들의 태도는 불쾌하고 오만했으며, 종교 집단에 속한 자가 가진 특유의 관대한 아량을 표출했다. 그들은 또한 오라에 내재된 위대한 실존적 진리를 알고 있다고 믿었으며, 그것은 사회를 움직이는 데막시아인에게는 너무나 위대하고도 고상한 믿음이라고 생각했다. 그들은 가치가 확실한 단 하나의 자연 자원에 그 이상의 영적인 무언가가 있다고 믿었다. 마치 오라가 신이나, 창조주나, 우주의 보편적인 영적 실체라도 되는 것처럼.

# 케인은 전에도 이런 종류의 광기를 본 적 있었다. 변방 행성에 거주하는 원시 사회에서는 나무나 자연, 생태계를 숭배했고 때로는 평범한 전투병기에 충격을 받고 이를 신으로 숭배하는 화물 숭배 신앙을 갖기도 했다.

# 어리석고 무지했다.

# 하지만 기사단은 조직적이고 호전적이며 은하계 전역에 걸친 지원망을 구축했다는 점에서 달랐다. 그들의 신념은 비정상적인 데다가 터무니없었지만, 그들의 미천한 추종자들은 제국으로부터 귀중한 오라 물자를 훔치거나 민간 재산을 강탈하면서까지 열정적으로 기사단을 신봉했다. 그들은 가장 질 나쁜 부류의 불온 분자들이었다.

# 케인은 죄수들이 붙잡혀 있는 동굴로 들어갔다. 그들은 언제나 그랬듯 사납고 완강하며 헌신적인 모습을 보였다. 자신의 신념을 위해 투쟁하는 자들이었다.

# 비참한 몰골의 죄수들이 자신을 보고 겁에 질린 것을 알아챈 케인은 만족감을 느꼈다. 이곳이 자신들의 종착지이며, 애처로운 믿음조차 더는 자신들을 지켜줄 수 없다는 사실을 알았으리라.

# "나는 오디널, 시이다 케인이다." 케인이 입을 열었다. "내가 가진 지위의 권한을 알고 있을 것이다. 보아하니 묵비권을 행사 중인 것 같은데."

# 죄수들은 움츠러들었다. 케인은 외계 종족이 적어도 여섯 종은 섞여 있음을 알아차렸다. 누구를 고를 것인가? 스콜도이는 어떨까? 아주 연약한 종족이었다.

# "체포당해 족쇄를 차고 있으면서도 두려워하지 않는 것 같군." 케인은 말을 이었다. "슬픈 일이야. 왜냐하면 내 경험상 굴복하는 것 말고 다른 선택지는 없거든. 그대들은 내 질문에 답하게 될 거다."

# "우리는 아무것도 말하지 않을 것이다." 몸집이 큰 코로바크가 으르렁거렸다.

# "그래? 왜지?" 케인이 물었다.

# "우리가 알고 있는 것은 당신과 같은 부류의 인간들에게는 어울리지 않기 때문이지."

# 일부가 동의하며 웅성거렸다. 그렇다면 코로바크인가, 케인은 생각에 잠겼다. 코로바크는 가장 덩치 큰 우두머리였다. 그에게 본때를 보여 주면 나머지는 알아서 꼬리를 내릴 것이다.

# 아니다. 그건 너무 쉽다.

# 케인은 미소를 지었다. "방금 내 질문에 대답했군, 코로바크."

# "난..."

# "난 질문했고, 자넨 대답했어." 케인은 말을 이었다. "어렵지 않았지? 그러니까 이건 일반적으로 자네가 답하기 어려운 문제는 아니었군? 그저 구체적일 뿐이었지."

# "난 네놈 장단을 맞춰 줄 생각이 없다." 코로바크가 딱 잘라 말했다.

# "그러면서 내가 자네 장단을 맞춰 줄 거라 생각한단 말이지. 아무래도 본론으로 들어가야 할 것 같군, 선생. 자넨 지금 협상할 처지가 아니야. 자, 그럼 시작해 보지. 난 이름을 원해. 자네가 가진 연락책과 변방 행성의 조력자들. 자네들을 이리로 데려온 두 명의 기사단원. 그들이 아이오난에 오기 전에 거래했던 사람들 말이야."

# 코로바크는 시선을 돌렸다.

# "그럼 첫 번째 이름부터 시작하지." 케인이 말했다.

# "우린 누군가가 '데려오지' 않았다." 코로바크는 중얼거렸다. "말해줄 것은 아무것도 없다."

# "정중히 부탁건대, 이름을 말해 주겠나?"

# 그는 그저 동굴 바닥만을 바라보고 있었다. 케인은 권총집의 걸쇠를 벗기고 광자 권총을 꺼내 들었다. 긴 크롬 총신이 황혼의 붉은 어스름을 받아 반짝였다. 케인은 엄지손가락으로 발동 장치를 쓸었다. 계기판이 발포 단계까지 올라가자 침울한 웅성거림이 들려왔다.

# "첫 번째 이름." 케인은 더욱 힘을 주어 말했다.

# 죄수는 고개를 저었다.

# 케인은 천천히 권총을 들어 올려 무릎을 꿇고 앉은 코로바크의 이마에 갖다 대었다. 공포에 휩싸인 죄수들이 속삭이는 소리가 들려 왔다. "첫 번째 이름." 케인은 다시 물었다.

# "원한다면 쏘게나." 코로바크가 바닥에서 눈을 떼지 않은 채 말했다. "그게 바로 제국주의자들의 사고방식이지. 협박하고, 짐승처럼 취급하고. 그러니 쏘게. 넌 절대로 원하는 걸 얻지 못할 테니. 나는 기사단의 축복을 받으며, 자네에게 저항했다는 만족감을 느끼며 오라 관문을 지날 걸세."

# "좋아." 케인이 말했다. "그럴 수도 있겠지. 하지만 이 놀이는 그렇게 끝나지 않아."

# 케인은 조준 대상을 바꾸었다. 광자 권총은 코로바크 뒤의 소녀를 겨눴다. 커다란 눈을 가진 진지한 표정의 묘한 분위기를 풍기는 소녀였다. 다른 종족들과 달리, 이 소녀는 케인과 그의 권총을 똑바로 바라봤다.

# "첫 번째 이름을 말해라, 코로바크. 아니면 이 아이가 대신 저승으로 가게 될 거야. 자넨 살아남겠지. 털끝 하나 다치지 않고 말이야. 축복이나 만족감이 아니라 이 아이의 죽음에 대한 죄책감이 네 몫이 될 거야."

# 코로바크는 근심스러운 눈으로 소녀를 바라보았다. "넌 그럴 수 없을 것이다. " 그가 쉰 목소리로 말했다.

# "오, 못 할 것도 없지." 케인이 말을 이었다. "한 명씩, 차례차례, 모든 명단과 질문에 대한 답을 얻어낼 때까지 몇 명이라도 말이지. 이건 아주 간단한 놀이야. 자네가 대답보다 목숨이 더 중요하다는 사실을 깨달으려면 시체가 몇 구나 필요할까. 하나? 셋? 열다섯? 백?"

# "어떻게 그렇게 잔인할 수가—"

# "이게 내 일이거든. 나도 좋아서 하는 일이 아니야. 내가 그저 질문하듯 누군가를 쉽게 죽인다고 생각하나? 자네, 오롯이 자네만이 이 일에 필요성을 부여한 거야. 나에게 선택지를 남겨 주지 않았지 않나. 솔직히 말하자면 자네가 어찌 그리 잔인한지 모르겠어. 단지 대답이 조금 느렸다고 해서 이 가여운 소녀가 죽어야 할 이유는 없지 않겠나.

# 코로바크는 마른침을 삼켰다. "나... 나는... 배신하지..."

# "좋아, 나는 원칙을 가진 사람을 존경하지." 케인은 한숨을 쉬었다. "원칙이란 위대해. 그 원칙으로 자신이 아닌 다른 사람이 죽는다면 더욱 그렇고."

# 그는 소녀를 바라보았다. 그 커다란 눈동자에는 기이하게도 아무런 두려움이 담겨 있지 않았다. 이렇게 침착한 죄수는 처음이었다. 거슬렸다. 그녀를 심문하고 싶은 기분이 들었다. 그녀에게 질문을 던지고 그녀가 아는 것 전부를 듣고 싶었다.

# 하지만 이 흥미는 막 생겨난 것이다. 케인은 그녀를 본보기로 선택했다. 여기서 물러나는 것은 약점을 노출하는 것이고, 그렇게 되면 남은 죄수들의 결의를 다지는 꼴밖에 더 되겠는가.

# 그렇지만...

# "동지들의 부족한 협조심을 네가 만회할 수 있단다." 케인은 소녀에게 직접 말을 걸었다. "그 정도는 해줄 수 있겠지. 네가 첫 번째 이름을 말해라. 대화로 참사를 막을 수 있다는 것을 이자에게 가르쳐 주면 나 또한 관용을 베풀어 주지."

# 소녀는 조용히 케인을 마주 보았다.

# "어서." 케인은 재촉했다. "첫 번째 이름을 말해. 이런 기회는 자주 오지 않아."

# "소나는 아무 말도 해줄 수 없다!" 코로바크가 흐느끼듯 쏘아붙였다.

# "아, 할 수 있고말고." 케인은 소녀의 눈을 바라보며 대답했다. "말하고 싶어 죽을 지경일걸. 소나라고? 그게 네 이름인가? 소나, 간단한 질문이야. 한 단어, 한 이름. 어려울 거 없어. 이름을 말해."

# 소녀는 아무런 반응도 보이지 않았다. 짜증이 곧 분노로 바뀌었지만 케인은 드러내지 않았다. 그는 충분히 절제했고 기회도 주었지만 소녀는 그를 웃음거리로 만들었다. 누구도 그를 이렇게 대우한 적 없었다.

# "소나, 실망스럽구나." 케인은 방아쇠를 당겼다.


# 폭발이 동굴을 휩쓸었다.

# 케인이 다시 제 발로 서기까지는 시간이 조금 걸렸다. 바깥의 흙먼지가 동굴로 쏟아져 들어왔고, 천장에서 파편들이 후드득 떨어져 내렸다. 그가 쏜 탄환은 폭발 충격으로 케인이 발을 헛디디는 바람에 크게 빗나갔다.

# 동굴 바깥에서 큰 소리가 두 번 더 울려 퍼졌다.

# "이동! 이동!" 나쿠리가 고함을 질렀다. 슬링트루퍼들은 앞다투어 출구로 뛰었다. 죄수들은 공포에 휩싸였다.

# 그 소녀를 제외하고는 말이다.

# "계속 감시해!" 케인은 솔리파스에게 소리치곤 출구를 향해 달려나갔다. 빛이 닿는 곳으로 빠져나가자 다시 선회한 소형 전투기가 상공에 나타났다. 나쿠리의 수송선 중 하나는 이미 파괴되어 불타고 있었다. 무광의 녹색 화살 전투기는 고원 위에 낮게 떠 포격을 가하고 있었다. 광자포 포드에서 칼날 같은 빛이 번쩍이자 두 번째 수송선이 폭발했고, 불기둥에 휩싸인 거대한 선체가 뒤집혀 추락하다 케인의 소형 데막스-3와 충돌했다.

# 나쿠리는 명령을 하달하고 있었고, 그의 슬링트루퍼들은 동굴 입구에 늘어섰다. 교전이 시작되자 빗발치는 탄환이 하늘을 가득 메웠다.

# "기다려!" 케인이 소리쳤다.

# "왜?" 나쿠리가 물었다.

# "사격을 중지해. 우릴 죽일 생각이었다면 진작 산을 무너뜨렸을 거야. 주의를 끌 속셈인 거지."

# "사격 중지!" 나쿠리가 명령했다.

# "함대에 연락해." 케인이 말했다. "현재 위치를 사수하고 구출하러 온다든가 하는 어리석은 짓은 하지 말라고 전해."

# "아슬아슬한 줄타기를 하는군, 친구."

# "언젠 안 그랬나. 자, 서둘러!"

# 케인은 나쿠리가 인터페이스를 작동하는 소리를 들으며 앞으로 걸어갔다. 불타는 함선의 잔해에서 나온 검은 연기가 수평으로 퍼져 나갔다. 지면에서 피어오른 아지랑이와 함께 연기가 물결쳤다. 얼굴에 열기가 느껴졌다.

# "와 보시지." 그가 중얼거렸다. "오라고, 어서..."

# 고원의 끄트머리에서 녹색 전투기가 모습을 드러냈다. 기체는 엔진을 아래로 분사해 제자리 비행 중이었다. 옅게 색을 입힌 창문에 햇빛이 반사됐다. 기체는 시야를 가로막은 짙은 연기를 뚫고 천천히 날아들었다. 왼쪽에서 두 번째 회색 전투기가 나타났다.

# 세 번째로 나타난 붉은색 전투기는 그들이 있는 고원의 중앙을 향해 곧장 다가왔다.

# 세 전투기가 20여 미터 떨어진 공중에서 멈추었다. "이런 제길." 나쿠리가 말했다. "신디케이트군."

# "그래." 케인이 대답했다. 그는 한눈에 온갖 잡동사니를 모아 개조한 공격용 함선을 알아보았다. 암시장에서 거래되는, 불법적이고 외계에서 가져온 무장 체계는 작은 선체와 어울리지 않게 비대칭적으로 거대했다. 기체 자체도 제국 통치기 이전의 기술을 사용한 구모델을 폐행성에서 인양해 와서 신디케이트의 솜씨 좋은 무기 설계자들이 개조한 것이 분명했다.

# 가장 거대한 붉은색 전투기의 동체 밑에는 포드가 달려 있었다. 밀수품인 은폐장 생성기였다. 미약한 반응 신호의 정체는 함선 한 대가 아니었다. '세 대'의 전투기들이 은폐장 안에서 밀집 대형으로 움직이며 하나의 유령 신호로 보였던 것이다. 질량이나 밀도에 대한 명확한 데이터가 없는 것은 놀랄 만한 일이 아니었다. 그들은 아마 하강 궤적을 타고 하나가 되어 움직이다가, 대기권에 진입하자마자 흩어졌을 것이다.

# '제법이군.' 케인은 생각했다. 밀수 경로 봉쇄나 함대를 이용한 수송로 차단 검문을 유유히 빠져나가는 전형적인 범죄 수법이다.

# 붉은색 함선이 약간 앞으로 다가왔다. 조종석이 불쑥 열렸다.

# "저 녀석은 나한테 맡겨." 나쿠리가 말했다.

# "내가 이야기해 볼게." 케인이 대답했다. "하지만 대원들을 대기시켜 둬. 순식간에 기습해서 끝내지 않으면 놈들이 이 구역 전체를 날려버릴 거야."

# 나쿠리는 고개를 끄덕였다. 그늘에서 나온 케인은 경사면을 타고 내려가 고원 꼭대기를 내리쬐는 강력한 햇빛 아래로 걸어 나왔다. 그는 머리를 치켜든 채 흙먼지 사이를 뚫고 맨 앞에 있는 전투기를 향해 성큼성큼 다가갔다.

# "이곳에 무슨 볼일이 있나?" 케인이 외쳤다.

# 붉은색 전투기의 조종석은 2인승이었다. 앞쪽에 앉아 있던 헬멧을 쓴 조종사가 총구로 케인을 겨누었다. 뒷좌석의 그림자는 자리에서 일어나 마스크를 벗었다. "있고말고요." 그가 말했다. "오디널을 만나게 될 거라고는 생각하지 못했지만 말이죠. 하지만 하루하루가 새롭고 흥분되는 게 인생 아니겠습니까?"

# 자고, 코런 자고였다. 은하계 변두리에서 활동하는 신디케이트의 요주 인물 중 하나 말이다.

# 케인의 인터페이스는 즉시 얼굴과 음성 인식으로 그를 식별했지만, 케인은 이미 그자에 대해 알고 있었다. 데막시아의 장교라면 모두 우주 곳곳에 붙은 수배 전단을 통해 자고의 얼굴을 알고 있었다. 그가 직접 모습을 드러내는 일은 거의 없었기 때문에 오랫동안 잡히지 않은 채 활동할 수 있었던 것이다.

# 무엇이 그렇게 중요했기에 그가 나타났던 것일까?

# "이거 영광이로군, 자고." 케인이 말했다. "이렇게 직접 만나게 되다니 말이야."

# 자고가 미소지었다. "제가 드릴 말씀입니다, 시이다 케인. 당신 이야기는 굉장히 많이 들었죠."

# "제국 기물에 상당한 피해를 입혔군." 케인이 불타는 잔해를 가리키며 말했다.

# "그저 저희의 결의를 보여드리고 싶었을 뿐입니다."

# "그렇다면 성공했네. 여긴 무슨 일로 왔지? 기사단과 그 추종자들을 만나러 왔나? 거래라도 하기로 한 건가?

# 자고는 진심으로 놀란 것처럼 보였다. "기사단이라고요? 도대체 제가 기사단에 무슨 볼일이 있겠습니까?"

# "여기서 만나기로 한 게 아니라고?"

# "아닙니다. 전혀 상관없는 일이죠."

# "그렇다면 뭐지?"

# "당신과 같은 이유일 것 같군요." 자고가 말했다. "오디널이 변방 행성까지 날아오는 게 흔한 일이 아니잖습니까?"

# "그렇지." 케인은 자신의 정보 부족을 감추기 위해 태연히 거짓말했다. "그렇다면 넌 어떻게 들었지?"

# 자고는 신중하게 대답했다. "아마 당신과 같은 출처겠죠."

# 케인은 그에게서 이상한 분위기를 읽었다. 코런 자고는 자신감 넘치고 거만하기로 소문이 자자한 자였다. 하지만 지금은 어딘가 불안해 보였다.

# "뭐, 그렇다면..." 케인은 자고의 어색함을 반영하듯 어깨를 으쓱했다. "자네도 알고 있군."

# "알죠." 자고는 솔직하게 고개를 끄덕였다. "정말 이상하지 않습니까? 그 목소리 말입니다. 마치 우주에서 들려오는 것처럼요. 전 그걸 가지러 이곳에 와야 한다는 걸 깨달았죠. 제 것이 되리라는 사실도요. 그러니 죄송하지만 오디널, 당신은 절 막을 수 없습니다. 그러니 그만 넘기고 물러나시죠. 전 무슨 일이 있어도 손에 넣고 말 겁니다. 저항하신다면... 뭐, 여길 쑥대밭으로 만들고 그걸 빼앗아 제국 함대가 눈치채기도 전에 사라질 겁니다."

# "터무니없는 소린 아니군."

# 터무니없는 소리였다. 자고는 위험인물이지만, 정신 이상자는 아니었다. 그의 전투기 세 대는 케인의 소규모 지상군을 압도할지 몰라도, '프랙털 쉬어'와 '젠틀 리마인더'는 신디케이트도 벌벌 떠는 로커스 아르마다의 전함이었다.

# 게다가 코런 자고가 몸소 행차했다니. 케인이 알고 있던 그의 전형적인 일 처리 방식이 아니었다. 뭔가 달랐다. 충동적이었다. 뭔가에 사로잡혀 있었다.

# 이것이 그의 약점을 드러냈다.

# 케인은 천천히 숨을 깊게 들이쉬어 정신을 맑게 했다. 그가 어떻게 오디널이 될 수 있었는지 보여줄 때였다.

# "이거, 꼼짝없이 당하게 생겼군." 케인은 중앙 행성 특유의 우아하고 과장된 동작으로 두 팔을 벌리며 말했다. 누구나 공식적인 항복 의사라는 것을 알아차릴 수 있는 의례적인 몸짓이었다. 그리고 한쪽 무릎을 꿇더니 어깨를 앞으로 숙이고 양팔을 내려놓으며 크게 절을 했다. 케인은 화려하게 장식된 창을 오른손에 쥐고 45도 각도로 날이 위를 향하도록 들었다. 군인으로서 경의를 뜻하는 각도였다. "이런 상황이라면 어쩔 수 없지."

# 케인은 따끔거리는 열기와 피어오른 연기를 느꼈다. 그에게 와 닿는 코런 자고의 시선도 느꼈다. 이렇게 수월하게 승리를 거두다니 그도 놀랐을 것이다.

# 케인은 강했다. 그의 기본적인 신체 능력은 지독한 훈련으로 단련되었고, 과학의 힘으로 강화되었다. 다른 모든 오디널과 마찬가지로 케인은 인간의 한계를 뛰어넘은 존재였다.

# 케인은 기다렸다. 자고의 입 밖으로 단 한 음절이 나올 때까지.

# "당신—"

# 케인은 무릎을 꿇은 상태에서 아래에서 위로 창을 던졌다. 창은 겨누어진 방향을 향해 일직선으로 날아갔다. 케인은 심지어 무릎을 꿇고 고개를 숙인 상태로 쳐다보지도 않았다.

# 힘차게 날아간 창은 공중에 떠 있던 붉은색 전투기의 은폐장 포드 바로 앞쪽을 꿰뚫었다. 넓은 창날은 기체를 뚫고 중앙부의 냉각기와 자세 제어 시스템, 조종석 바닥을 지나 코런 자고에게까지 날아갔다.

# 창이 마침내 멈추었을 때, 전투기는 꼬치에 꿰인 모양을 하고 있었다. 창 자루가 기체의 바닥을 뚫고 올라와 자고를 공격한 모양새였다.

# 자고는 등받이가 높은 의자에 꼿꼿이 고정되어 있었다. 생기가 사라진 그의 얼굴은 놀란 표정을 짓고 있었다.

# 한순간의 정적이 지나자 기체 내부 시스템이 터지고 파괴된 붉은색 전투기가 거칠게 요동쳤고, 압력이 조정되지 않는 엔진에서 굉음이 났다. 신디케이트의 조종사들이 상황을 파악하고 반응을 보이기까지 조금 시간이 걸렸다.

# 하지만 이미 늦었다. 나쿠리가 대기하고 있었던 것이다. 케인이 창을 던지는 것을 보자마자 나쿠리는 신호를 보냈고, 슬링트루퍼들은 일제 사격을 개시했다. 회색과 녹색 전투기에 광자탄이 빗발쳤다. 첫 번째 기체는 그 자리에서 집중포화를 받아 산산이 부서진 채로 추락했다. 드라이브 코어가 폭발하면서 조각난 기체 파편들이 불덩이처럼 이곳저곳으로 튀어 나갔다.

# 케인은 무릎을 굽힌 자세를 역이용하여 뛰어올랐다. 요동치는 붉은색 전투기는 그의 머리에 닿을 정도로 낮게 떠 있었고, 그는 오른쪽 날개를 뛰어넘었다. 전투기가 빙빙 돌자 조종사가 기체를 제어하기 위해 고군분투했다. 왼쪽 날개 끝이 지면에 부딪히며 사방에 자갈을 흩날렸고, 추력 장치는 사막의 모래폭풍처럼 거센 먼지를 일으켰다.

# 불안정한 기체에 올라탄 케인은 열려 있는 조종실에 접근했다. 자고는 그 자리에 고정되어, 먼 곳을 응시한 채 기체가 덜컥거릴 때마다 함께 들썩일 뿐이었다. 조종사는 조종 장치와 씨름하느라 바빠서 다른 일에 신경 쓸 겨를이 없었다.

# 나쿠리의 슬링트루퍼들은 계속해서 공격을 쏟아부었지만, 녹색 전투기는 제압하기 어려웠다. 광자 에너지를 흡수하는 특수 차폐막 때문이었다. 작은 빛들이 깜빡이더니 기체 앞쪽에서 끈적이는 뿌연 기체가 뿜어져 나왔다. 이어 날카로운 굉음이 나면서 무기 포드가 열렸고, 연쇄 폭발이 슬링트루퍼들의 진형을 덮쳤다.

# 나쿠리가 즉시 흩어지라는 명령을 내리기도 전에 부하 두 명이 그 자리에서 산화해 버렸다. 고도를 높인 전투기는 달아나는 슬링트루퍼들을 사냥하기 시작했다. 그들의 지상사격은 무방비 상태인 항공기를 상대할 때만 효과가 있었다.

# 기습 공격의 이점은 이미 사라져 버렸다.

# 케인은 한 손으로 좌석에 앉은 조종사를 잡고 밖으로 내던졌다. 왼쪽 날개에 부딪혀 튀어 오른 그는 놀라서 비명을 지르며 지면으로 떨어졌다.

# 케인은 조종실의 덮개를 붙잡고 조종석 안으로 뛰어들었다. 그는 인터페이스를 통해 안정장치 제어기가 완전히 망가졌음을 알아차렸다. 그가 던진 창이 일부 주요 시스템까지 꿰뚫었던 것이다. 케인은 추진기와 정지해 버린 엔진 포트를 빠르게 조정한 후 조종실 덮개를 열어둔 채로 전투기를 전진시켰다. 전투기는 땅에 닿을 정도로 낮게 털털거리며 앞으로 움직였다.

# 녹색 전투기는 산비탈에 폭격을 가하고 있었다. 주무장 포드를 열고 산 전체를 날려버릴 준비를 하고 있었다. 조종 레버를 끌어당기던 케인은 전투기의 사격 통제 시스템을 작동하여 주포를 장전하고 눈앞의 녹색 전투기를 조준했다.

# 그는 전투기를 향해 광자포를 퍼부었다. 그 반동으로 불안정한 기체가 거칠게 요동쳤다. 술에 취한 것처럼 휘청이며 제 위치를 벗어났고, 마지막 포격은 궤적을 크게 벗어나 예광탄처럼 산 너머의 하늘을 밝혔다.

# 하지만 첫 번째 포격은 명중했다. 녹색 전투기는 먼저 후미를, 다음에는 엔진 하나를 잃었다. 조종사는 균형을 잡으려고 애썼지만 후미가 폭발하자 결국 기체 전체가 허공에서 산산이 부서졌다. 기체가 상승하며 엄청난 화염과 파편들이 솟구쳤고, 곧이어 곤두박질치다 땅에 처박혔다.

# 그 폭발로 모래 위에 충격파가 발생하자 열기에 녹아내린 거대한 구덩이가 생겼다.

# 케인은 타고 있던 전투기가 추락하지 않도록 고도를 유지하느라 사투를 벌였다. 제어판에 오작동 경고들이 한꺼번에 울렸다. 하나씩 전력을 차단하며 기체를 진정시켰고, 전투기는 결국 튀어 오르다가 한쪽 날개를 모래에 파묻으며 주욱 미끄러졌다.

# 케인은 모든 전력을 차단했다. 모래가 튀어 오르며 방풍창과 기체를 타닥타닥 두드렸다. 케인은 조종석에서 몸을 일으켜 자고의 놀란 표정을 마지막으로 한번 흘겨본 후, 지상으로 뛰어내렸다.

# 케인이 걸어가는 동안 기체 내부에서 무언가가 발화되었고 곧 큰 불길이 치솟았다. 한 남자를 품고 타오르는 붉은색 전투기를 뒤로한 채 케인은 나쿠리에게 다가갔다.

# 나쿠리는 슬링트루퍼들을 불러모으고 있었다. 그리고는 충격과 감탄이 뒤섞인 눈빛으로 케인을 쳐다보았다. "제정신이 아니군." 그의 목소리는 단호했다.

# "글쎄." 케인이 대답했다. "하지만 어쩌다 이 지경이 됐는지, 아주 오래전 일처럼 느껴지는군."
# """
# a=json.dumps(a,ensure_ascii=False)[1:-1]
# print(a)

In [30]:
a = relationship_select("케인",15,"ssssssss")
for i in a:
    print(i)
b = Relationship(a)

({'자르반 4세': {'관계': '친구', '대상이름': '케인', '상세내용': "나이가 비슷하고 친구 같은 사이. 케인에게 제국의 정책에 대한 속내를 털어놓기도 함. 케인을 '사냥개'라고 비유하며 그의 충성도와 호전성을 시험하길 좋아함."}},)
({'베키드': {'관계': '상관', '대상이름': '케인', '상세내용': "케인을 '오디널 각하'라고 부르며 존칭을 사용함. 케인의 지시에 따라 말을 꺼냄."}},)
({'콜라': {'관계': '상관', '대상이름': '케인', '상세내용': '케인이 인터페이스를 통해 이름을 확인한 슬링트루퍼 중 한 명.'}},)
({'스픽스': {'관계': '상관', '대상이름': '케인', '상세내용': '케인이 인터페이스를 통해 이름을 확인하고 동등하게 대하려는 인물'}},)
({'리고': {'관계': '상관', '대상이름': '케인', '상세내용': '케인이 인터페이스를 통해 이름을 확인하고 동등하게 대하려는 인물'}},)
({'바서르': {'관계': '상관/보고 대상', '대상이름': '케인', '상세내용': '케인에게 상황을 보고하고 그의 지시를 받는다. 케인의 신중함 때문에 그의 명령에 따른다. 케인이 바서르 함장에게 항로 변경을 요청하라고 키일로에게 지시함.'}},)
({'키일로': {'관계': '오랜 친구', '대상이름': '케인', '상세내용': '케인이 추측하기로는 상대방을 배려하거나 프로토콜을 따르는 것으로 보임. 케인의 책상을 부숨. 케인에게 대련을 제안함. 케인의 추적기를 떼어줌.'}},)
({'키일로': {'관계': '동료', '대상이름': '케인', '상세내용': '케인에게 무장 필요 여부를 묻고, 케인의 임무에 동행하려 함. 케인이 혼자 가겠다고 하자 따름. 케인이 웃음소리를 들었다고 했을 때 자신은 웃지 않았다고 답함.'}},)
({'슬링트루퍼': {'관계': '협력자', '대상이름': '케인', '상세내용': '케인과 함께 작전을 수행하며 그의 지시를 따른다.'}},)
({'코로바크': {